In [57]:
import findspark
import re
#findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName="first")

In [4]:
sc.getConf().getAll()

[('spark.app.id', 'local-1689508779525'),
 ('spark.driver.port', '54091'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'Amir.mshome.net'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'first'),
 ('spark.app.startTime', '1689508777558')]

In [5]:
fruit_rdd = sc.parallelize(["Apple", "Orange", "Kiwi"])
fruit_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [6]:
fruit_rdd.collect()
# Если rdd слишком большой - то это убьет наш клиентский сервер, поскольку все данные со всех экзекютеров 
#  поедут на наш сервер, пожтому коллект надо использовать только если данных не много

['Apple', 'Orange', 'Kiwi']

In [7]:
fruit_rdd.getNumPartitions() # сколько у нас сплитов

8

In [8]:
fruit_rdd.glom().collect()

[[], [], ['Apple'], [], [], ['Orange'], [], ['Kiwi']]

In [12]:
stop_words_rdd = sc.textFile("./stop_words_english.txt")
print(stop_words_rdd.getNumPartitions())
print(stop_words_rdd.count())
print(stop_words_rdd.take(10))

2
851
['able', 'about', 'above', 'abroad', 'according', 'accordingly', 'across', 'actually', 'adj', 'after']


In [13]:
stop_words_rdd = stop_words_rdd.repartition(10)
print(stop_words_rdd.getNumPartitions())
print(stop_words_rdd.count())
print(stop_words_rdd.take(10))

10
851
['cant', "can't", 'caption', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', "c'mon"]


In [14]:
simple_rdd = sc.textFile("simple_text.txt")
simple_rdd

simple_text.txt MapPartitionsRDD[21] at textFile at <unknown>:0

In [16]:
print(simple_rdd.getNumPartitions())
print(simple_rdd.count())

2
3


In [17]:
simple_rdd.first()  # Первый элемент из rdd

'a 1 first'

In [18]:
favorite_letters_rdd = simple_rdd.map(lambda x: x.split())
favorite_letters_rdd.first()

['a', '1', 'first']

In [20]:
simple_rdd.is_cached # закешировано ли rdd

False

In [21]:
# simple_rdd.cache() - кешировать данные, чтобы не перезапускать расчеты заново
# simple_rdd.unpersist() - очистить кеш

In [40]:
wiki_rdd = sc.textFile("wiki_simple.txt")

In [41]:
wiki_rdd = (
    wiki_rdd
    .map(lambda x: x.split("\t", 1)) # get list of words
    .map(lambda pair: pair[1].lower()) # lower words in context
    .flatMap(lambda content: content.split())) # split to words

In [42]:
wiki_count_rdd = (
    wiki_rdd
    .map(lambda x: (x, 1))
    .reduceByKey(lambda x, y: x+y)
)

In [43]:
wiki_count_rdd.cache()

PythonRDD[63] at RDD at PythonRDD.scala:53

In [44]:
%%time
print(wiki_count_rdd.is_cached)
print(wiki_count_rdd.first())
print()

True
('in', 4)

Wall time: 7.32 s


In [45]:
%%time
print(wiki_count_rdd.is_cached)
print(wiki_count_rdd.first())
print()

True
('in', 4)

Wall time: 1.87 s


In [46]:
wiki_count_rdd.unpersist()

PythonRDD[63] at RDD at PythonRDD.scala:53

In [47]:
%%time
print(wiki_count_rdd.is_cached)
print(wiki_count_rdd.first())
print()

False
('in', 4)

Wall time: 2.01 s


## Spark Broadcast

In [49]:
stop_words_rdd = sc.textFile("stop_words_english.txt")
stop_words_broadcast = sc.broadcast(stop_words_rdd.collect()) # помещаем в оперативную память, раскидываем 
                                                             #  наш датасет на все executors

In [56]:
wiki_count_rdd.filter(lambda x: x[0] not in stop_words_broadcast.value).takeOrdered(10, key = lambda x: -x[1])

[('quantum', 6),
 ('protons', 5),
 ('atom', 4),
 ('atoms', 4),
 ('neutrons', 3),
 ('early', 3),
 ('particle', 3),
 ('nucleus', 2),
 ('physics', 2),
 ('chemical', 2)]

## Spark Joins and PairRDD

Можно реализовать mapside join, у нас весь RDD закешированный находится на оперативной памяти на executors, и мы туда бродкастим наш датасет и на месте производим join 

In [58]:
stop_words_rdd = sc.textFile("stop_words_english.txt")
stop_words_broadcast = sc.broadcast(stop_words_rdd.collect())

In [61]:
triple_dataset = wiki_count_rdd.map(lambda x: (x[0][0], x[0], x[1]))
triple_dataset.first()

('i', 'in', 4)

In [62]:
favorite_letters_rdd.first()

['a', '1', 'first']

In [63]:
favorite_letters_rdd.join(triple_dataset).take(5) # сджойнили по первой колонке, остались вторые колонки,
                                                  # третьи колонки потерялись

[('b', ('2', 'based')),
 ('b', ('2', 'basic')),
 ('b', ('2', 'by')),
 ('b', ('2', 'bound')),
 ('b', ('2', 'be'))]

In [64]:
# чтобы не потерять колокник запокуем в лист вторую и третью колонки
triple_dataset = wiki_count_rdd.map(lambda x: (x[0][0], [x[0], x[1]]))
triple_dataset.first()

('i', ['in', 4])

In [65]:
favorite_letters_rdd.join(triple_dataset).take(5)

[('b', ('2', ['based', 1])),
 ('b', ('2', ['basic', 1])),
 ('b', ('2', ['by', 3])),
 ('b', ('2', ['bound', 1])),
 ('b', ('2', ['be', 1]))]

[More examples of RDD functions](https://sparkbyexamples.com/spark/spark-pair-rdd-functions/)

In [66]:
sc.stop()